# Generate a Project Template

## Imports

In [17]:
from typing import List

import subprocess
import json
import re
from pathlib import Path
from dataclasses import dataclass, InitVar, field

import pandas as pd


In [2]:
class ProjectException(Exception):
    '''Base Exception for projects module.'''
    pass


class AnacondaException(ProjectException):
    '''Errors related to `conda` calls.'''
    pass


class PathException(ProjectException):
    '''Errors related to directory and file specification.'''
    pass

## Utility Functions

### Console command processing

In [3]:
def error_check(output: subprocess.CompletedProcess, error_type: Exception,
                error_msg: str):
    '''Check for errors in subprocess output.

    if a `CalledProcessError` error occured raise the error, with the
    error_msg text.
    Args:
        output (subprocess.CompletedProcess): subprocess output to be tested.
        error_type (Exception): The type of exception to raise if an error
            occurs.
        error_msg (str): The Error message to include if required.
    Raises:
        error_type: If subprocess generated an error.
    '''
    try:
        output.check_returncode()
    except subprocess.CalledProcessError as err:
        msg = '\n'.join([error_msg, output.stderr.decode()])
        raise AnacondaException(msg) from err


In [4]:
def console_command(cmd_str: str, error_type: Exception, error_msg: str)->str:
    '''Run a system console command.

    Run the command. Check for errors and raise the appropriate error if
    necessary.
    If no error, return the output from the result of running the console
    command.

    Args:
        cmd_str (str): The console command to execute.
            command.
        error_type (Exception): The type of exception to raise if an error
            occurs.
        error_msg (str): The Error message to include if required.

    Returns:
        str: The log output from running the console command.
    '''
    cmd_list = cmd_str.split(' ')
    output = subprocess.run(cmd_list, shell=True, capture_output=True)
    # check for errors
    error_check(output, error_type, error_msg)
    cmd_log = output.stdout.decode()
    return cmd_log


In [5]:
def add_folder(base: Path, name: str)->Path:
    '''Make a subdirectory under base.

    Generates a path to the subdirectory.
    If it doesn't exist create it.

    Args:
        base (Path): The top folder (parent to the subdirectory).
        name (str): The name of the subdirectory.

    Returns:
        Path: The path to the subdirectory.
    '''
    sub_folder = base / name
    if not sub_folder.exists():
        sub_folder.mkdir()
    return sub_folder


### Anaconda utilities

#### Start Anaconda

In [6]:
def start_anaconda():
    anaconda_launcher = r'C:\ProgramData\Anaconda3\Scripts\activate.bat'
    output = console_command(anaconda_launcher, AnacondaException,
                             'Unable to start Anaconda')
    return output


#### Activate an environment

In [7]:
def activate_environment(new_env):
    start_anaconda()
    anaconda_activate = f'conda activate {new_env}'
    output = console_command(anaconda_activate, AnacondaException,
                             f'Unable to activate environment {new_env}')
    return output


#### List Anaconda environments

In [8]:
def list_environments():
    conda_envs = r'conda info --envs --json'
    env_list = console_command(conda_envs, AnacondaException,
                               'Unable to get Anaconda environments')

    return env_list


In [9]:
def list_environments():
    env_pattern = re.compile(
        r'(?P<name>'  # Start of *name* group.
        r'[a-z0-9_]'  # Name begins with letter, number or _.
        r'.+?'        # All text until 2 or more spaces are encountered
        r')'          # End of *name* group.
        r'[ *]{2,}'   # 2 or more spaces or * in a row.
        r'(?P<path>'  # Start of *path* group.
        r'[A-Z]:'     # Drive letter, followed by a :.
        r'[^\r\n]*'   # Remaining text before the end of the line.
        r')',         # End of *path* group.
        flags=re.IGNORECASE)

    start_anaconda()
    conda_envs = r'conda info --envs'
    env_list = console_command(conda_envs, AnacondaException,
                               'Unable to get Anaconda environments')
    if env_list:
        env_info = {name: path for name, path in env_pattern.findall(env_list)}
    else:
        env_info = None
    return env_info


In [10]:
def save_env_specs(new_env: str, env_folder: Path):
    '''Store *spec* and *.yml* for the environment.

    Args:
        new_env (str): Name of Conda environment.
        env_folder (str): Path to the project's environment folder.
    '''
    start_anaconda()

    spec_file = env_folder / f'{new_env}_specfile.txt'
    save_spec_cmd = ' '.join([f'conda list --explicit',
                              f'--name {new_env} >',
                              str(spec_file)])
    console_command(save_spec_cmd, AnacondaException,
                    f'Error saving {new_env} environment spec file.')

    full_yml_file = env_folder / f'{new_env}_FULL.yml'
    save_full_yml = ' '.join([f'conda env export --name {new_env} --file',
                              str(full_yml_file)])
    console_command(save_full_yml, AnacondaException,
                    f'Error saving {new_env} full .yml file.')

    history_yml_file = env_folder / f'{new_env}.json'
    save_yml = ' '.join(['conda env export --from-history --json'
                         f' --name {new_env}',
                         '>',
                          str(history_yml_file)])
    console_command(save_yml, AnacondaException,
                    f'Error saving {new_env} history .json '
                    f'to {str(history_yml_file)}.')


#### Remove an Anaconda environment

In [11]:
def remove_environment(del_env):
    start_anaconda()
    output = console_command('conda deactivate', AnacondaException,
                             'Error deactivating environment.')
    delete_cmd = f'conda remove -y --name {del_env} --all'
    uninstall_log = console_command(delete_cmd, AnacondaException,
                                    f'Unable to delete environment {del_env}')
    return uninstall_log


#### Save environment specs

(EDW_QA) C:\Users\smoke>conda env export -h
usage: conda-env-script.py export [-h] [-c CHANNEL] [--override-channels] [-n ENVIRONMENT | -p PATH] [-f FILE]
                                  [--no-builds] [--ignore-channels] [--json] [-v] [-q] [--from-history]

Export a given environment

optional arguments:
  -h, --help            Show this help message and exit.
  -c CHANNEL, --channel CHANNEL
                        Additional channel to include in the export
  --override-channels   Do not include .condarc channels
  -f FILE, --file FILE  File name or path for the exported environment. Note: This will silently overwrite any
                        existing file of the same name in the current directory.
  --no-builds           Remove build specification from dependencies
  --ignore-channels     Do not include channel names with package names.
  --from-history        Build environment spec from explicit specs in history

Target Environment Specification:
  -n ENVIRONMENT, --name ENVIRONMENT
                        Name of environment.
  -p PATH, --prefix PATH
                        Full path to environment location (i.e. prefix).

Output, Prompt, and Flow Control Options:
  --json                Report all output as json. Suitable for using conda programmatically.
  -v, --verbose         Can be used multiple times. Once for detailed output, twice for INFO logging, thrice for DEBUG
                        logging, four times for TRACE logging.
  -q, --quiet           Do not display progress bar.

examples:
    conda env export
    conda env export --file SOME_FILE


In [19]:
save_env_specs('Standard', Path(r'C:\temp'))


### Other Install Commands

#### Install Jupyter Kernel 

In [12]:
def install_kernel(new_env):
    '''Install the ipython kernel in the new environment.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    start_anaconda()
    activate_environment(new_env)
    kernel_cmd = ''.join([
        'python -m ipykernel install --user --name',
        new_env,
        '--display-name',
        f'"Python ({new_env})"'
        ])
    kernel_install_log = console_command(
        kernel_cmd,
        AnacondaException,
        f'Unable to install ipython kernel in "{new_env}"!'
        )
    return kernel_install_log


#### Configure xlwings

In [13]:
def configure_xlwings(new_env: str)->str:
    '''Install and configure xlwings binaries for environment.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    start_anaconda()
    activate_environment(new_env)
    xlwings_license = 'xlwings license update -k noncommercial'
    license_log = console_command(
        xlwings_license, AnacondaException,
        f'Unable to set xlwings license in environment {new_env}')
    xlwings_addin = 'xlwings addin install'
    addin_log = console_command(
        xlwings_addin, AnacondaException,
        f'Unable to install xlwings addin in environment {new_env}')
    xlwings_config = 'xlwings config create --force'
    config_log = console_command(
        xlwings_config, AnacondaException,
        f'Unable to configure xlwings in environment {new_env}')
    xlwing_log = '\n\n'.join([license_log, addin_log, config_log])
    return xlwing_log


### Class to hold information on folders related to the package.

In [14]:
# FIXME This class build the folder when the instance is defined.
# This should happen at a later time.
@dataclass
class FolderItem:
    '''Manage folders related to the package.'''
    folder_name: str
    tag: str = ''
    add_to_path: bool = True
    folder_path: Path = None
    base_path: InitVar[Path] = None

    def __post_init__(self, base_path: Path):
        '''Manage defaults for folder settings.

        If not defined, base_path defaults to the current working
        directory.  If it doesn't exists the folder is created.

        If tag is not given use uppercase of folder name for tag.
        '''
        if not base_path:
            base_path = Path.cwd()
        self.set_path(base_path)
        self.set_tag()

    def set_path(self, base_path: Path):
        '''If missing, Set the folder_path to a subdirectory of base_path.

        Generates a path to the subdirectory.
        If it doesn't exist create it.

        Args:
            base_path (Path): The top folder
                (parent to the subdirectory).
        '''
        if not self.folder_path:
            sub_folder = base_path / self.folder_name
            if not sub_folder.exists():
                sub_folder.mkdir()
            self.folder_path = sub_folder

    def set_tag(self):
        '''If tag not given use uppercase of folder name.'''
        if not self.tag:
            self.tag = self.folder_name.upper()


### Package Specifier

In [18]:
@dataclass
class PackageSpec:
    '''Package name and version specification.'''
    name: str
    version: str = ''  # Version number with optional logic
    # if only version given use '=' for logic
    # Version with logic examples:
    # Equal to 2.5: "=2.5"
    # Higher than 2.5: ">2.5"
    # Lower than 3.2: "<3.2"
    # Higher than 2.5 AND Lower than 3.2: ">2.5,<3.2"
    # Equal to 2.5 OR Equal to 2.5 OR "[version='2.5|3.2']"
    version_types = ('=', '<', '>', '[')
    default_relation = '='

    def __post_init__(self):
        '''Manage defaults for version.

        If version does not start with one of self.version_types, add
        self.default_relation to the beginning of the version string.
        '''
        if self.version:
            if not self.version.startswith(self.version_types):
                self.version = self.default_relation + self.version

    def specifier(self)->str:
        '''Return the full package specifier for conda Install.

        Combines package name and version specifier.
        '''
        return self.name + self.version

    def __eq__(self, package_name):
        '''True if the name of the PackageSpec matches package_name.
        '''
        return self.name == package_name


# Done To Here

## Project Parameters

### Packages
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-hnrj{background-color:#C6E0B4;color:#ffffff;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-psg0{background-color:#70AD47;color:#ffffff;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-0blg{color:#ffffff;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-225c{background-color:#C6E0B4;color:#ffffff;text-align:left;vertical-align:top}
.tg .tg-7803{color:#ffffff;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-psg0"><span style="font-weight:700;text-decoration:none;background-color:#70AD47">Package</span></th>
    <th class="tg-psg0"><span style="font-weight:700;text-decoration:none;background-color:#70AD47">Category</span></th>
    <th class="tg-psg0"><span style="font-weight:700;text-decoration:none;background-color:#70AD47">Description</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">numpy</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">Fundamental   package for array computing in Python</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pandas</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">Powerful   data structures for data analysis, time series, and statistics</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">matplotlib</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">Python   plotting package</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">pysimplegui</td>
    <td class="tg-7803">GUI</td>
    <td class="tg-7803">Python   GUIs for Humans. </td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">scipy</td>
    <td class="tg-7803">Data Analysis</td>
    <td class="tg-7803">Fundamental   algorithms for scientific computing in Python</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">shapely</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">Manipulation   and analysis of geometric objects</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">xlwings</td>
    <td class="tg-7803">Excel</td>
    <td class="tg-7803">Interact   with Excel from Python and vice versa.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">openpyxl</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Excel</span></td>
    <td class="tg-7803">A   Python library to read/write Excel 2010 xlsx/xlsm files</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">xlrd</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Excel</span></td>
    <td class="tg-7803">Library for   developers to extract data from Microsoft Excel (tm) .xls spreadsheet files</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">pyodbc</td>
    <td class="tg-7803">Data Analysis</td>
    <td class="tg-7803">DB   API module for ODBC</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pydicom</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">A   pure Python package for reading and writing DICOM data</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pymedphys</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">Medical   Physics library</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">networkx</td>
    <td class="tg-7803">graphs and networks</td>
    <td class="tg-7803">Python   package for creating and manipulating graphs and networks</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">graphviz</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">graphs and networks</span></td>
    <td class="tg-7803">Simple   Python interface for Graphviz</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pydot</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">graphs and networks</span></td>
    <td class="tg-7803">Python   interface to Graphviz's Dot</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">imageio</td>
    <td class="tg-7803">Imaging</td>
    <td class="tg-7803">Library   for reading and writing a wide range of image, video, scientific, and   volumetric data formats.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pillow</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Imaging</span></td>
    <td class="tg-7803">Python   Imaging Library (Fork)</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">scikit-image</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Imaging</span></td>
    <td class="tg-7803">Image   processing in Python</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">scikit-learn</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Data Analysis</span></td>
    <td class="tg-7803">A set of   python modules for machine learning and data mining</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">spyder</td>
    <td class="tg-7803">Development</td>
    <td class="tg-7803">The   Scientific Python Development Environment</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">jupyter</td>
    <td class="tg-7803">jupyter</td>
    <td class="tg-7803">Jupyter   metapackage. Install all the Jupyter components in one go.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">jupyterlab</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">jupyter</span></td>
    <td class="tg-7803">JupyterLab   computational environment</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">ipykernel</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">jupyter</span></td>
    <td class="tg-7803">IPython   Kernel for Jupyter</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">nb_conda_kernels</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">jupyter</span></td>
    <td class="tg-7803">Package   for managing conda environment-based kernels inside of Jupyter</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">spyder-kernels</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">jupyter</span></td>
    <td class="tg-7803">Jupyter   kernels for Spyder's console</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">ipython</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">jupyter</span></td>
    <td class="tg-7803">IPython:   Productive Interactive Computing</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">pywin32</td>
    <td class="tg-7803">File interaction</td>
    <td class="tg-7803">Python   for Window Extensions</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">docx2txt</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">File interaction</span></td>
    <td class="tg-7803">A   pure python-based utility to extract text and images from docx files.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">zipfile2</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">File interaction</span></td>
    <td class="tg-7803">An   improved ZipFile class.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pdfminer.six</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">File interaction</span></td>
    <td class="tg-7803">Pdfminer.six   is a community maintained fork of the original PDFMiner.</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">requests</td>
    <td class="tg-7803">Web Site Interaction</td>
    <td class="tg-7803">Python   HTTP for Humans.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">beautifulsoup4</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Web Site Interaction</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Beautiful Soup is a library that makes   it easy to scrape information from web pages.</span></td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">bs4</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Web Site Interaction</span></td>
    <td class="tg-7803">Dummy   package for Beautiful Soup</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">soupsieve</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Web Site Interaction</span></td>
    <td class="tg-7803">A modern CSS   selector implementation for Beautiful Soup.</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">pylint</td>
    <td class="tg-7803">style checker</td>
    <td class="tg-7803">python   code static checker</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">mypy</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">style checker</span></td>
    <td class="tg-7803">Optional   static typing for Python</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pydocstyle</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">style checker</span></td>
    <td class="tg-7803">Python   docstring style checker</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pyflakes</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">style checker</span></td>
    <td class="tg-7803">passive   checker of Python programs</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">black</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">style checker</span></td>
    <td class="tg-7803">The   uncompromising code formatter.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">autopep8</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">style checker</span></td>
    <td class="tg-7803">A   tool that automatically formats Python code to conform to the PEP 8 style   guide</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">pep8</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">style checker</span></td>
    <td class="tg-7803">Python style   guide checker</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">doxygen</td>
    <td class="tg-7803">Documentation</td>
    <td class="tg-7803">Doxygen is a documentation   generator and static analysis tool for software source trees.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">sphinx</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Documentation</span></td>
    <td class="tg-7803">Python   documentation generator</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">docutils</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Documentation</span></td>
    <td class="tg-7803">Docutils --   Python Documentation Utilities</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">pyinstaller</td>
    <td class="tg-7803">Development</td>
    <td class="tg-7803">PyInstaller   bundles a Python application and all its dependencies into a single package.</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">setuptools</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Development</span></td>
    <td class="tg-7803">Easily   download, build, install, upgrade, and uninstall Python packages</td>
  </tr>
  <tr>
    <td class="tg-hnrj"><span style="font-weight:700;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
    <td class="tg-225c"><span style="font-weight:400;text-decoration:none;background-color:#C6E0B4"> </span></td>
  </tr>
  <tr>
    <td class="tg-0blg">fastapi</td>
    <td class="tg-7803">Servers</td>
    <td class="tg-7803">FastAPI   framework, high performance, easy to learn, fast to code, ready for   production</td>
  </tr>
  <tr>
    <td class="tg-0blg"><span style="font-weight:700;text-decoration:none">uvicorn</span></td>
    <td class="tg-7803"><span style="font-weight:400;text-decoration:none">Servers</span></td>
    <td class="tg-7803">The   lightning-fast ASGI server.</td>
  </tr>
</tbody>
</table>

**Uncomment packages to include them**

In [19]:
primary_packages = [
    #PackageSpec('numpy'),
    PackageSpec('pandas'),
    #PackageSpec('matplotlib'),

    #PackageSpec('pysimplegui'),

    #PackageSpec('scipy'),
    #PackageSpec('shapely'),

    PackageSpec('xlwings'),
    #PackageSpec('openpyxl'),
    #PackageSpec('xlrd'),

    #PackageSpec('pyodbc'),
    #PackageSpec('pydicom'),
    #PackageSpec('pymedphys'),

    #PackageSpec('networkx'),
    #PackageSpec('graphviz'),
    #PackageSpec('pydot'),

    #PackageSpec('imageio'),
    #PackageSpec('pillow'),
    #PackageSpec('scikit-image'),
    #PackageSpec('scikit-learn'),

    #PackageSpec('requests'),
    #PackageSpec('beautifulsoup4'),
    #PackageSpec('soupsieve'),

    #PackageSpec('docx2txt'),
    #PackageSpec('pdfminer.six'),
    #PackageSpec('pywin32'),
    #PackageSpec('zipfile2'),
    ]

tool_packages = [
    #PackageSpec('fastapi'),
    #PackageSpec('uvicorn'),

    #PackageSpec('doxygen'),
    #PackageSpec('sphinx'),
    #PackageSpec('docutils'),

    #PackageSpec('jupyter'),
    #PackageSpec('nb_conda_kernels'),

    #PackageSpec('spyder'),
    #PackageSpec('spyder-kernels'),

    #PackageSpec('pyinstaller'),
    #PackageSpec('setuptools'),
    #PackageSpec('autopep8'),
    #PackageSpec('black'),
    #PackageSpec('mypy'),
    #PackageSpec('pydocstyle'),
    #PackageSpec('pyflakes'),
    #PackageSpec('pylint'),
    #PackageSpec('pep8'),
    ]
package_list = primary_packages + tool_packages

## Specify project related paths
- The top level project path should already exist and should be given the tag 'WORKSPACE'
- `folder_path` must be specified for 'WORKSPACE'
- An 'ENVIRONMENT' must be defined.
- There shouldn't be duplicate tags.
- If `tag` is not given `folder_name.upper()` will be used as the `tag`
- When `folder_path` is given, the `folder_name` is ignored.  
- If `folder_path` is not given, a folder with the name `folder_name` will be created directly under `base_path`.  
- If `add_to_path` is True *(the default)*, the folder path will be added to the python search path.

#### Example folders:

| folder_name | folder_path                              | tag         | add_to_path |
|-------------|------------------------------------------|-------------|-------------|
| Todoist     | ~\Python\Projects\ToDoIst Projects\Turkey| WORKSPACE   | True        |
| environment | None                                     | ENVIRONMENT | False       |
| src         | None                                     | SOURCE      | True        |
| Old files   | None                                     | IGNORE      | False       |
| data        | None                                     | DATA        | True        |
| reference   | None                                     | REFERENCE   | False       |
| sectionary  | ..\\..\sectionary package\src'           | PACKAGE     | True        |

### Primary project path

In [27]:
project_folder = Path.cwd()
project_drive = project_folder.drive


### Additional package path

In [28]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()


#### Folder Parameters

In [29]:
new_env = 'Turkey'

In [30]:
folder_list = [FolderItem(new_env,       tag='WORKSPACE',                      folder_path=project_folder),
               FolderItem('environment', tag='ENVIRONMENT', add_to_path=False,   base_path=project_folder),
               FolderItem('src',         tag='SOURCE',                           base_path=project_folder),
               FolderItem('data',        tag='DATA',                             base_path=project_folder),
               FolderItem('reference',   tag='REFERENCE',   add_to_path=False,   base_path=project_folder),
               FolderItem('sectionary',                                        folder_path=sectionary_path)]

folders = pd.DataFrame(folder_list)


In [31]:
folders

,folder_name,tag,add_to_path,folder_path
0,Turkey,WORKSPACE,True,c:\Users\gsalomon\OneDrive - Kingston Health S...
1,environment,ENVIRONMENT,False,c:\Users\gsalomon\OneDrive - Kingston Health S...
2,src,SOURCE,True,c:\Users\gsalomon\OneDrive - Kingston Health S...
3,data,DATA,True,c:\Users\gsalomon\OneDrive - Kingston Health S...
4,reference,REFERENCE,False,c:\Users\gsalomon\OneDrive - Kingston Health S...
5,sectionary,SECTIONARY,True,C:\Users\gsalomon\OneDrive - Kingston Health S...


In [33]:
folders[folders.tag == 'WORKSPACE']

,folder_name,tag,add_to_path,folder_path
0,Turkey,WORKSPACE,True,c:\Users\gsalomon\OneDrive - Kingston Health S...


In [ ]:
def make_project_folder(folder_list: List[FolderItem], project_name: str)->Path:
        '''Create the project folder if it does not exist.
        '''
        if isinstance(folder_list, Path):
            if not starting_folder.is_dir():
                raise PathException(
                    'The specified project folder is not a directory')
            starting_folder = folder_list
        elif isinstance(folder_list, list):
            folders = pd.DataFrame(folder_list)
            workspace = folders[folders.tag == 'WORKSPACE']
            starting_folder = workspace.folder_path

        if not starting_folder.exists():
            if starting_folder.parent.exists():
                starting_folder = add_folder(starting_folder.parent,
                                             project_name)
        elif starting_folder.name == project_name:

            self.project_folder = add_folder(self.project_folder, self.name)
            if self.project_folder.parent.exists():

In [30]:
folders


,folder_name,tag,add_to_path,folder_path
0,Todoist,WORKSPACE,True,D:\OneDrive - Queen's University\Python\Projec...
1,environment,ENVIRONMENT,False,D:\OneDrive - Queen's University\Python\Projec...
2,src,SOURCE,True,D:\OneDrive - Queen's University\Python\Projec...
3,data,DATA,True,D:\OneDrive - Queen's University\Python\Projec...
4,reference,REFERENCE,False,D:\OneDrive - Queen's University\Python\Projec...
5,sectionary,SECTIONARY,True,D:\OneDrive - Queen's University\Python\Projec...


### Class to hold information on project parameters.

In [ ]:
@dataclass
class ProjectDef:
    '''Information on project parameters.'''
    # Project Name and folders
    name: str
    folder_list: List[FolderItem] = field(default_factory=list)
    # Optional Conda environment name and packages to include.
    env_name: str = ''
    package_list: List[PackageSpec] = field(default_factory=list)
    # Project Name and Python Version
    python_version: str = '3.11'
    install_xlwings: bool = True
    install_jupyter: bool = True


    add_to_path: bool = True
    base_path: InitVar[Path] = None

    def __post_init__(self, base_path: Path):
        '''Manage default parameters.

        Create the project folder if it does not exist

        If xlwings is not in the list of packages do not install xlwings.
        If jupyter is not in the list of packages do not install jupyter.
        '''
        if not self.project_folder.exists():
            if self.project_folder.parent.exists():
                self.project_folder = add_folder(self.project_folder.parent,
                                                 self.name)
        elif self.project_folder.name == self.name:

            if not self.project_folder.is_dir():
                raise PathException(
                    'The specified project folder is not a directory')
            self.project_folder = add_folder(self.project_folder, self.name)
            if self.project_folder.parent.exists():
        if 'xlwings' not in package_list:
            self.install_xlwings = False
        if 'jupyter' not in package_list:
            self.install_jupyter = False

        self.project_drive = project_folder.drive

    @property
    def packages(self):
        '''Generate a string of packages to install.
        '''
        packages = ' '.join(self.package_list)
        return packages

    def set_tag(self):
        '''If tag not given use uppercase of folder name.'''
        if not self.tag:
            self.tag = self.folder_name.upper()


## Package Creation

### Create the new environment

In [9]:
print(f'Creating environment for {new_env}')
env_create_cmds = ['conda', 'create', '-y', '--name', new_env, f'python={python_version}', '--json']
output = subprocess.run(env_create_cmds,shell=True, capture_output=True)
# check for errors
try:
    output.check_returncode()
except subprocess.CalledProcessError as err:
    print(f'Unable to create new environment "{new_env}" with python={python_version}!')
    print(output.stderr.decode())
    raise err

install_log = output.stdout.decode()
#print(install_log)
logging_dict = json.loads(install_log)


Creating environment for Todoist


### Identify the Conda Environment path

In [13]:
env_path = Path(logging_dict['prefix'])


### Install the Packages

In [15]:
# Activate the new environment
env_activate_cmd = ['conda', 'activate', new_env]
output = subprocess.run(env_activate_cmd,shell=True, capture_output=True)
# check for errors
try:
    output.check_returncode()
except subprocess.CalledProcessError as err:
    print(f'Unable to activate new environment "{new_env}"!')
    print(output.stderr.decode())
    raise err


In [18]:
# Group Install the desired packages
env_install_cmds = ['conda', 'install', '-y', '--prefix', str(env_path), '--json'] + package_list
output = subprocess.run(env_install_cmds,shell=True, capture_output=True)
# check for errors
try:
    output.check_returncode()
except subprocess.CalledProcessError as err:
    print(f'Unable to install requested packages in "{new_env}"!')
    print(output.stderr.decode())
    raise err
else:
    pkg_install_log = output.stdout.decode()
    pkg_install_dict = json.loads(pkg_install_log)


### Write the Log Data generated during the Conda Environment install

In [32]:

log_file = env_folder / f'{new_env}CreationLog.txt'
pkg_table_file = env_folder / f'{new_env}Packages.csv'

install_log = install_log + '\n' + pkg_install_log
log_file.write_text(install_log)

pkg_items = logging_dict['actions']['LINK'] + pkg_install_dict['actions']['LINK']
pkg_table = pd.DataFrame(pkg_items)
pkg_table.to_csv(pkg_table_file)


In [7]:
if install_jupyter:


In [33]:
if install_xlwings:


In [34]:
print(f'Storing environment for {new_env}')


## Build Boilerplate Files

### Build Jupyter lab Batch file

In [41]:

jupter_batch_file = env_folder / f'JupyterLab ({new_env}).bat'
jupyter_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    'jupyter-lab'
    ])
jupter_batch_file.write_text(jupyter_batch)


198

### Build VS Code Batch file

In [42]:
vscode_batch_file = env_folder / f'VS Code ({new_env}).bat'
vscode_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    f'code {new_env}.code-workspace'
    ])
vscode_batch_file.write_text(vscode_batch)


214

### Build .env file

In [14]:
folders


,folder_name,tag,add_to_path,folder_path
0,Todoist,WORKSPACE,True,D:\OneDrive - Queen's University\Python\Projec...
1,environment,ENVIRONMENT,False,D:\OneDrive - Queen's University\Python\Projec...
2,src,SOURCE,True,D:\OneDrive - Queen's University\Python\Projec...
3,data,DATA,True,D:\OneDrive - Queen's University\Python\Projec...
4,reference,REFERENCE,False,D:\OneDrive - Queen's University\Python\Projec...
5,sectionary,PACKAGE,True,D:\OneDrive - Queen's University\Python\Projec...


In [17]:
folders[folders.add_to_path]


,folder_name,tag,add_to_path,folder_path
0,Todoist,WORKSPACE,True,D:\OneDrive - Queen's University\Python\Projec...
2,src,SOURCE,True,D:\OneDrive - Queen's University\Python\Projec...
3,data,DATA,True,D:\OneDrive - Queen's University\Python\Projec...
5,sectionary,PACKAGE,True,D:\OneDrive - Queen's University\Python\Projec...


In [ ]:
folders[folders.tag=='ENVIRONMENT'].folder_path


In [51]:
env_file = env_folder / '.env'

env_paths = '\n'.join('PYTHONPATH=${}' for name, path_str in path_dict.items())
path_setter = '\n'.join('PYTHONPATH="${' + name.upper() + '};${PYTHONPATH}"'
                        for name in path_dict.keys())
env_text = '\n'.join([env_paths, path_setter])
env_file.write_text(env_text)


183

### Build Code-workspace from templatestr(env_path)

In [15]:
data_dir = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")

worspace_template = data_dir / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [16]:
worspace = worspace_template.read_text()


In [43]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }


In [68]:
str(env_path).replace('\\', '/')


'C:/Users/smoke/.conda/envs/Todoist'

In [63]:
env_manage_base = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")
code_workspace_template = env_manage_base / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [69]:
template = code_workspace_template.read_text()


In [70]:
template = template.replace('%env_path%', str(env_path).replace('\\', '/'))


In [35]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")

env_folder = add_folder(project_folder, 'environment')
source_folder = add_folder(project_folder, 'src')

project_drive = project_folder.drive


In [43]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()


In [43]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }


In [ ]:
template.replace('%analysis.extraPaths%', str(text))


In [ ]:
template.replace('%autoComplete.extraPaths%', str(text))


# Misc Reference